<a href="https://colab.research.google.com/github/AlessandroTaglieri/Project_Bioinformatics/blob/main/First%20project/BI_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#comment if not running on colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import libraries
import pandas as pd
import os
import numpy as np
import collections

In [ ]:
#change project folder to match yours
project_folder ='/content/drive/MyDrive/BioinfoProject/Project_Bioinformatics/First_project/'
#project_folder ='/'

In [ ]:
#change dataset folder to match yours
dataset_folder = os.path.join(project_folder,'dataset/')

In [ ]:
#only in local
dataset_folder="dataset/"

# 1.1

In [ ]:
#download curated gene-diesase associtaions from https://www.disgenet.org/downloads and covnert tsv in csv

data = pd.read_csv(dataset_folder+ '1.1/curated_gene_disease_associations.csv')
#get only data for intereted gene id C0043094
gene_disease = data.loc[data['diseaseId'] == 'C0043094']

#get list og gene symbols
list_geneSymbol = gene_disease['geneSymbol']


#download hgnc dataset from https://www.genenames.org/download/statistics-and-files/
hgnc = pd.read_csv(dataset_folder+ "1.1/hgnc_complete_set.txt", sep='\t', dtype = 'unicode')

#get data (symbol,name,entrez_id,unirpot_idasdn status) for only genes stored in previous list
genes_diseases_hgnc = hgnc.loc[hgnc['symbol'].isin(list_geneSymbol)][['symbol','name','entrez_id','uniprot_ids','status']]

#get only approved data
genes_diseases_hgnc_app = genes_diseases_hgnc.loc[genes_diseases_hgnc['status']=='Approved']


#get list ofuniprotidthat we'll useto find gene descriptionin Uniprot
list_uniprotID = genes_diseases_hgnc_app['uniprot_ids']

#write in a file alll uniprot ids. This is necessary because we'll download only necessary description for these genes from uniprot 
with open(dataset_folder+ '1.1/uniprot_ids.txt', 'w') as f:
    for item in list_uniprotID:
        f.write("%s\n" % item)
    f.close()

#download data containing description for every gene. We used txt file for download them
gene_description = pd.read_csv(dataset_folder+ '1.1/uniprot-yourlist%3AM20201123A94466D2655679D1FD8953E075198DA817123DO.csv')

#rename column for the function of the gene
gene_description=gene_description.rename(columns={"yourlist:M20201123A94466D2655679D1FD8953E075198DA817123DO":"uniprot_ids"})

#merge two dataset. One containing all info about gene and the second one conatining description about their functions
final_genes_deseas = pd.merge(gene_description, genes_diseases_hgnc_app, on='uniprot_ids')
#rename column
final_genes_deseas=final_genes_deseas.rename(columns={"Function [CC]":"function"})
#get only first part of gene description
final_genes_deseas['Description']= final_genes_deseas.function.str[10:150]
#delete not necessary columns
final_genes_deseas = final_genes_deseas.drop(['function','Entry','status'], axis = 1) 
final_genes_deseas = final_genes_deseas[['symbol','uniprot_ids','name','entrez_id','Description']]

In [ ]:
#fix column names and save xlsx file
final_genes_deseas.rename(columns={"symbol":"Gene symbol","uniprot_ids":"Uniprot AC","name":'Protein name','entrez_id':"Entrez Gene ID"})
final_genes_deseas.to_excel(dataset_folder+ '1.1/1.1-gene_deseas_associations.xlsx')
final_genes_deseas.to_csv(dataset_folder+ '1.1/1.1-gene_deseas_associations.csv')

In [ ]:
final_genes_deseas

In [ ]:
## To save seed id and symbols correspondance
seed_symbols, seed_ids = final_genes_deseas['symbol'].values ,final_genes_deseas['entrez_id'].values
seed_symbols_to_ids = {}
seed_ids_to_symbols = {}
for key,value in zip(seed_symbols,seed_ids):
  seed_symbols_to_ids[key] = value
  seed_ids_to_symbols[value] = key


def seed_symbol_to_id(symbol):
  return seed_symbols_to_ids.get(symbol)

def seed_id_to_symbol(id):
  return seed_ids_to_symbols.get(id)


# 1.2

# Get biogrid data

In [ ]:

#only for Ale
#biogrid_datafile = '/Users/digitalfirst/Desktop/BI/BIOGRID-ORGANISM-Homo_sapiens-4.2.191.tab3.txt'
#read biogrid dataset 
biogrid_datafile = dataset_folder+'1.2/BIOGRID-ORGANISM-Homo_sapiens-4.2.191.tab3.txt'

In [ ]:

biogrid_data = pd.read_csv(biogrid_datafile,sep='\t')
#biogrid_data.info()

In [ ]:
# Filter data to only physical interactions and Homo sapiens genes
biogrid_data = biogrid_data[biogrid_data['Experimental System Type']=='physical']
biogrid_data = biogrid_data[biogrid_data['Organism Name Interactor A']=='Homo sapiens']
biogrid_data = biogrid_data[biogrid_data['Organism Name Interactor B']=='Homo sapiens']

In [ ]:
biogrid_data.head(5)

#### Mapping protein Id to symbols


In [ ]:
biogrid_ids_A,biogrid_symbols_A  = biogrid_data['Entrez Gene Interactor A'].values ,biogrid_data['Official Symbol Interactor A'].values
biogrid_ids_B,biogrid_symbols_B  = biogrid_data['Entrez Gene Interactor B'].values ,biogrid_data['Official Symbol Interactor B'].values

symbols_to_ids = {}
ids_to_symbols = {}
for key,value in zip(biogrid_symbols_A,biogrid_ids_A):
  symbols_to_ids[key] = value
  ids_to_symbols[value] = key

for key,value in zip(biogrid_symbols_B,biogrid_ids_B):
  symbols_to_ids[key] = value
  ids_to_symbols[value] = key 
  

In [ ]:
def symbol_to_id(symbol):
  id = symbols_to_ids.get(symbol)
  if id ==None:
    return seed_symbol_to_id(symbol)
  return id

def id_to_symbol(id):
  symbol = ids_to_symbols.get(id)
  if symbol == None:
    return seed_id_to_symbol(id)
  return symbol

def symbol_link_to_id_link(link):
  return (symbol_to_id(link[0]),symbol_to_id(link[1]))


##Get all nodes involved in interactions 


---



In [ ]:
gene_A_list = biogrid_data.iloc[:,1].values
gene_B_list = biogrid_data.iloc[:,2].values
A_B_list = np.append(gene_A_list,gene_B_list)
A_set = set(gene_A_list)

B_set =  set(gene_B_list)
A_B_set = set(A_B_list)

In [ ]:
genes_biogrid = np.array([id_to_symbol(x) for x in A_B_set if str.isdigit(str(x))])
genes_biogrid

##Get all interactions 



In [ ]:
interactions = biogrid_data.iloc[:,1:3].values
#interactions =np.array([[a,b] for a,b in interactions if(str.isdigit(str(a)) & str.isdigit(str(b)))])
interactions =np.array([[id_to_symbol(a),id_to_symbol(b)] for a,b in interactions ])
interactions.shape

In [ ]:
interactions

# Seed gene list 

In [ ]:
seed_genes_ids  =  final_genes_deseas['entrez_id'].values
seed_genes_ids = [id_to_symbol(s) for s in seed_genes_ids]


#Now we build a dictionary of all interactions

In [ ]:
all_interactions_dict = collections.defaultdict(set)
for a,b in interactions:
  all_interactions_dict[a].add(b)
  all_interactions_dict[b].add(a)

# Interactions including seed genes 

In [ ]:
seed_interactions_dict = collections.defaultdict(set)
for key in seed_genes_ids:
  seed_interactions_dict[key] = all_interactions_dict[key]

# Protein interacting with seed genes

In [ ]:
non_seed_genes_ids = set()
# from seed genes interactions dictionary, we retrieve all genes interacting with them
# performing union of all set 
for key in seed_interactions_dict.keys():
  non_seed_genes_ids = non_seed_genes_ids.union(seed_interactions_dict[key]) 

#We remove all seed genes included in the set
non_seed_genes_ids = np.array([x for x in non_seed_genes_ids if x not in seed_genes_ids])

# There we get only non seed gene which interacted with at least one seed gene
len(non_seed_genes_ids)

##Then we create a dictionary for interactions of non seed genes involved with at least one seed gene

In [ ]:
non_seed_interactions_dict = collections.defaultdict(set)

#looping on non seed gend ids 
for key in non_seed_genes_ids:
  # we retrieve interactions from all interactions dictionary 
  related_genes = all_interactions_dict[key]

  related_non_seed_genes = [x for x in related_genes if x not in seed_genes_ids]

  #we add it tho the seed genes iteractions dictionary 
  non_seed_interactions_dict[key] = set(related_non_seed_genes)


#Lets put all this in a data frame

In [ ]:
values = []
for a ,b in seed_interactions_dict.items():
  values.append([a,'T',','.join(b)])

for a ,b in non_seed_interactions_dict.items():
  values.append([a,'F',','.join(b)])


df = pd.DataFrame(values,columns=['gene','seed_gene_involved','interactions_seeds'])

In [ ]:
df.sample(10)

#1.2 b - Store data in csv format

In [ ]:
filename = dataset_folder + '1.2/interactions.csv'
df.to_csv(filename, sep='\t',index=False)

#1.2 c - Summary

In [ ]:
n_seed_disgenet = len(set(seed_genes_ids))
n_seed_biogrid = len(set([gene for gene in genes_biogrid if gene in seed_genes_ids]))
n_interacting_genes = len(set(genes_biogrid))
n_interactions = biogrid_data.shape[0]

In [ ]:
table = [[n_seed_disgenet,n_seed_biogrid,n_interacting_genes,n_interactions]]
summary = pd.DataFrame(table, columns=['N° seed disgenet','N° seed biogrid','Interacting genes','Interactions'])
summary

#1.3 Arrange interaction data

In [ ]:
genes_interactions = biogrid_data.values
indexes = []
i =0
for row in genes_interactions:
  check = str(row[7]) in seed_genes_ids
  check_2 = str(row[8]) in seed_genes_ids
  #  unwanted row were filtered out in biogrid data
  if check_2  and check:
    indexes.append(i)
    i= i+1
  



#1.3 a - seed genes interactome: interactions that involve seed genes only

In [ ]:
only_seed_interactions = biogrid_data.iloc[indexes,[7,8,23,26]]
only_seed_data = only_seed_interactions.values
columns = ['interactor A gene symbol',' interactor B gene symbol', 'interactor A Uniprot AC', 'interactor B Uniprot AC']
only_seed_interactions_df = pd.DataFrame(only_seed_data,columns=columns)

In [ ]:
#save gene seed interactions into csv file
filename = dataset_folder + '1.3/gsi.csv'
only_seed_interactions_df.to_csv(filename, sep='\t',index=False)


In [ ]:
only_seed_interactions_df

##1.3-b All proteins interacting with at least one seed gene

In [ ]:
proteins_interactions = biogrid_data.values
indexes = []
i =0
for row in proteins_interactions:
  check_1 = str(row[7]) in seed_genes_ids
  check_2 = str(row[8]) in seed_genes_ids
  # check_3 = row[35] =='Homo sapiens' # not necessary anymore because unwanted row were filtered out
  # check_4 = row[36] =='Homo sapiens'
  if (check_2  or check_1) :
    indexes.append(i)
    i= i+1

In [ ]:
all_proteins_interactions = biogrid_data.iloc[indexes,[7,8,23,26]]
all_proteins_data = all_proteins_interactions.values
columns = ['interactor A gene symbol',' interactor B gene symbol', 'interactor A Uniprot AC', 'interactor B Uniprot AC']
all_proteins_interactions_df = pd.DataFrame(all_proteins_data,columns=columns)

In [ ]:
all_proteins_interactions_df

##Save interactome

In [ ]:
#save inteactome disease csv
filename = dataset_folder + '1.3/interactome.csv'
all_proteins_interactions_df.to_csv(filename, sep='\t',index=False)



#2 - Data Analysis

In [ ]:
!pip install networkx  # to install package needed for the network

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

### Build a network object from our data 

In [ ]:
bio_graph = nx.Graph()  # create the empty grap object 

In [ ]:
links = all_proteins_interactions_df.iloc[:,:2].values# get all links 
edges = [tuple(link) for link in links ] # transform links in tuple for nx graph object 

In [ ]:
bio_graph.add_edges_from(edges)

### Visualize graph 

In [ ]:
nx.draw_networkx(bio_graph, 
                       with_labels=False,
                       node_size=10,
                       node_color=('b'), 
                       edge_color=('g'),
                 

                       )
plt.show()

##2.1 Main network measures

#### a - global measures

In [ ]:
avg_degree = sum(nx.average_neighbor_degree(bio_graph).values())/ nx.number_of_nodes(bio_graph)
avg_clus_coef = nx.average_clustering(bio_graph)


In [ ]:
avg_paths = []
radiuses = []
diameters = []
for cc in nx.connected_components(bio_graph):
  g = bio_graph.subgraph(cc)   
  avg_paths.append(nx.average_shortest_path_length(g))
  radiuses.append(nx.radius(g))
  diameters.append(nx.diameter(g)) 


In [ ]:
avg_path_lengths = np.mean(avg_paths)
radius = np.max(radiuses)
diameter = np.max(diameters)

In [ ]:
print(f"Number of nodes               : {nx.number_of_nodes(bio_graph)}")
print(f"Number of Links               : {nx.number_of_edges(bio_graph)}")
print(f"Number of connected components: {nx.number_connected_components(bio_graph)}")
print(f"Number of isolated nodes      : {nx.number_of_isolates(bio_graph)}")
print(f"Average path length           : {avg_path_lengths}")
print(f"Average Degree                : {avg_degree}")
print(f"Average custering coefficient : {avg_clus_coef}")
print(f"Network radius                : {radius}")
print(f"Network diameter              : {diameter}")
print(f"Network centralization              : {'#TODO'}")

#### b- LCC Measures

In [ ]:
bio_graph_lcc = max(nx.connected_components(bio_graph), key=len)
bio_graph_lcc = bio_graph.subgraph(bio_graph_lcc)

In [ ]:
nx.draw_networkx(bio_graph_lcc, 
                 with_labels=False,
                 node_size=5,
                 node_color=('b'), 
                 edge_color=('g')
)
plt.show()

In [ ]:
#calculate newtork centralization and network degree
degs = list(bio_graph_lcc.degree(bio_graph_lcc.nodes))
vals=[]
d = nx.density(bio_graph_lcc)
for k,v in degs:
    vals.append(v)
mm=max(vals)
num_n = float(len(bio_graph_lcc.nodes()))
centralization = (num_n/(num_n-2))*(mm/(num_n-1)-d)
avg_degree = sum(nx.average_neighbor_degree(bio_graph_lcc).values())/ nx.number_of_nodes(bio_graph_lcc)
avg_path_lengths = nx.average_shortest_path_length(bio_graph_lcc)

##### i - Global Measures

In [ ]:
print(f"Number of nodes               : {nx.number_of_nodes(bio_graph_lcc)}")
print(f"Number of Links               : {nx.number_of_edges(bio_graph_lcc)}")
print(f"Number of connected components: {nx.number_connected_components(bio_graph_lcc)}")
print(f"Number of isolated nodes      : {nx.number_of_isolates(bio_graph_lcc)}")
print(f"Average path length           : {avg_path_lengths}")
print(f"Average Degree                : {avg_degree}")
print(f"Average custering coefficient : {nx.average_clustering(bio_graph_lcc)}")
print(f"Network radius                : {nx.radius(bio_graph_lcc)}")
print(f"Network diameter              : {nx.diameter(bio_graph_lcc)}")
print(f"Network centralization        : {centralization}")

##### ii - Local Measures

 ###### Node degree



In [ ]:
g_degree = nx.degree(bio_graph_lcc)
degree_df = pd.DataFrame(g_degree, columns=['Node','Degree'])
#degree_df.head(10)

###### Betweenness centrality

In [ ]:
g_b_centrality = nx.betweenness_centrality(bio_graph_lcc)
#g_b_centrality

###### Eigenvector centrality


In [ ]:
g_e_centrality = nx.eigenvector_centrality(bio_graph_lcc)
#g_e_centrality

###### Closeness centrality

In [ ]:
g_c_centrality = nx.closeness_centrality(bio_graph_lcc)
#g_c_centrality

##### Show all local measures

In [ ]:
nodes = list(bio_graph_lcc.nodes)
nodes_degree = list(degree_df['Degree'].values)
b_centrality = list(g_b_centrality.values())
e_centrality = list(g_e_centrality.values())
c_centrality = list(g_c_centrality.values())


###### Ratio Betweenness/Node degree

In [ ]:
ratio_b_d = np.array(b_centrality)/np.array(nodes_degree)

In [ ]:
columns = ['nodes','Degree','Betweeness Centrality','Eigen centrality','closeness centrality','Ratio']
data = [nodes,nodes_degree,b_centrality,e_centrality,c_centrality,ratio_b_d]
data = np.array(data).T;
df = pd.DataFrame(data, columns=columns)
df.sort_values(by='Betweeness Centrality',ascending=False)
df['Betweeness Centrality','Eigen centrality','closeness centrality','Ratio'] = df['Betweeness Centrality','Eigen centrality','closeness centrality','Ratio'].round(5)
df.head(20)

#### 2.2 Clustering methods for disease modules discover

In [ ]:
!pip install markov_clustering[drawing]

In [ ]:
import markov_clustering as mc

In [ ]:
#  get the adjacency matrix (in sparse form)
matrix = nx.to_scipy_sparse_matrix(bio_graph)

In [ ]:
result = mc.run_mcl(matrix)           # run MCL with default parameters
clusters = mc.get_clusters(result)   # each element in cluster is represented by internal ID number

###### clusters with at least 10 nodes

In [ ]:
# we retrieve cluster with at least 10 elements 
above_10_clusters = [cluster for cluster in clusters if len(cluster)>=10]

# We convert back the IDs to genes symbols
nodes_list = list(bio_graph.nodes)
symbols_clusters = [tuple([nodes_list[i] for i in cluster]) for cluster in above_10_clusters]

#### hypergeometric test

In [ ]:
import random
from collections import Counter
from scipy.stats import hypergeom


In [ ]:
collection =  list(all_proteins_interactions_df.iloc[:,:2].values.flatten())
collection_set = set(collection)

In [ ]:
N_seed_genes = n_seed_biogrid # number of seedgenes in iogrid
N_all_genes = len(genes_biogrid)
N_non_seed_genes = N_all_genes - N_seed_genes
prob_seed = N_seed_genes / N_all_genes
prob_seed

In [ ]:
sample_size_max = 10



In [ ]:
probs = {}
hpd = hypergeom(N_all_genes,N_seed_genes,sample_size_max)
for x in range(1,sample_size_max):
  probs[x] = hpd.pmf(x)

probs

For p < 0.05 we have to find sample with at least 1 seed gene

#### Check cluster that contains the over represented Nodes

In [ ]:
putative_disease_modules = []
for cluster in symbols_clusters:
  s = len(set(cluster).intersection(set(seed_genes_ids)))
  if s > 0:
    putative_disease_modules.append(cluster)

#putative_disease_modules

In [ ]:
module_data = []
columns = ['module ID', 'no.of seed genes in the module',
           'total no. of genes in each module', 'seed gene IDs', 'all gene IDs in the module', 'p-value']
for id,module in enumerate(putative_disease_modules):
  module_id = f'MOD_{id}'
  n_seed_gene = len(set(module).intersection(set(seed_genes_ids)))
  module_size = len(module)
  seed_genes = ','.join(set(module).intersection(set(seed_genes_ids)))
  all_genes = ','.join(module)
  p_value = hypergeom(N_all_genes,N_seed_genes,module_size).pmf(n_seed_gene)

  module_data.append([module_id,n_seed_gene,module_size,seed_genes,all_genes,p_value])

  

In [ ]:
moduledf = pd.DataFrame(module_data,columns=columns)
moduledf